In [ ]:
from Crypto.Util.number import bytes_to_long
from random import randint

#===== Ring setup =======
n=2^9 #ring dim
qc=2^22 #ctx mod
qm=2^8 #ptx mod
P=PolynomialRing(Integers(qc),'x')
x=P.gen()
Pq=P.quotient(x^n-1,'y')
y=Pq.gen()


#====== RLWE =======
def gen_RLWE(m,s):
    a=Pq.random_element()
    mask= Pq.random_element()*(y-1)
    e=sum([y^i*(randint(0,2^4)-2^3) for i in range(n)])
    b=s*a+m+mask+qm*e
    return(b,a)


#====== Flag encryption =======
pad = hex(bytes_to_long(b'ptm{N0t_4ll_R1n9s_4r3_G00d_4_RLWE!!}'))[2:]
#pad = hex(bytes_to_long(b'ptm{???????????????????????????????}'))[2:]
newpad = [pad[i:i+12] for i in range(0,len(pad),12)]
RLWEs=[]
for i in newpad:
    s=Pq.random_element()
    for j in [i[k:k+2] for k in range(0,len(i),2)]:
        b,a=gen_RLWE(int(j,16),s)
        RLWEs.append([b,a])
        


In [ ]:
#==== Attack =======
from tqdm import tqdm
from Crypto.Util.number import long_to_bytes


def RLWE_to_LWE(b):
    return(b.lift()(1))


def LWE_dec(b,a,s):
    return( mod(int(b-a*s),qm))

#READ RLWEs from a file!!!

res=[]
LWEs=[[RLWE_to_LWE(poly) for poly in RLWE] for RLWE in RLWEs]
same_key_samples=6
for k in range(0,len(LWEs),same_key_samples):
    sk_attempts,res_attempts=[],[]
    for sk_attempt in tqdm(range(qc)):
        res=[mod(b-sk_attempt*a,qc) for (b,a) in LWEs[k:k+6]]
        absres=[[abs(int(x)),abs(int(x)-qc)] for x in res]
        checks=[ min(x[0],x[1])<2^17 for x in absres]
        if all(checks)==True: sk_attempts.append(sk_attempt)
    if sk_attempts==[]:
        print("ERROR")
        continue
    for sk_attempt in sk_attempts:
        res_attempts.append([long_to_bytes(int(LWE_dec(b,a,sk_attempt))) for (b,a) in LWEs[k:k+6]])
    for R in res_attempts:
        print(R)
    res.append(res_attempts)